# Mediods Analysis

This experiment has the purpose of analysing the medoid of some of the clusters defined in the previous experiment. According to it, the best number of cluster is 82 regarding the square of the 2-norm distance, and 802 regarding the silhouette score. With 802 clusters, the silhouette score was 0.1, a considerably low value. Therefore, we won't consider it in the next experiments, implying that 82 is the best number of clusters.

In [1]:
#!pip install numpy==1.15 wordcloud

In [2]:
import sys
import os
sys.path.append('../')
from src import reader as r
from src import visualization as v

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
%matplotlib notebook
import numpy as np
import pandas as pd
import sklearn
print(sklearn.__version__)
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances

0.20.0


In [4]:
X = r.readBOW()
print(X[:10])
print(X.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(13229, 1203)


In [5]:
pca = PCA(n_components = 0.85, svd_solver='full')
X = pca.fit_transform(X)
npX = X

In [6]:
from importlib import reload
reload(r)
news = r.readNews()
print(news.head(10))
print(news.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
Read 13229 lines
                   id                    publish_date  \
0  576880531301801984  Sat Mar 14 23:00:11 +0000 2015   
1  576820122666471424  Sat Mar 14 19:00:08 +0000 2015   
2  576744652717461504  Sat Mar 14 14:00:15 +0000 2015   
3  576736754436304896  Sat Mar 14 13:28:52 +0000 2015   
4  576736614766010368  Sat Mar 14 13:28:18 +0000 2015   
5  576548368740052992  Sat Mar 14 01:00:17 +0000 2015   
6  576518190286536704  Fri Mar 13 23:00:22 +0000 2015   
7  576494177971732480  Fri Mar 13 21:24:57 +0000 2015   
8  576472874946433024  Fri Mar 13 20:00:18 +0000 2015   
9  576464606551490560  Fri Mar 13 19:27:27 +0000 2015   

                                       headline_test  
0  An abundance of online info can turn us into e...  
1  A plant-based diet that incorporates fish may ...  
2  It doesn't take much to damage your hearing at...  
3  RT @CNN: Forever young? Discover this island’s...

In [7]:
from MulticoreTSNE import MulticoreTSNE as TSNE

smp_sz = X.size
tsne_bow = TSNE(n_components=2, perplexity=10, verbose=True, n_jobs=-1)#500
tsne_bow_result = tsne_bow.fit_transform(X)

## Computing the Medoids
In here, we compute the medoids as the closest point to the centroids of the clusters.

In [8]:
best_K = 61
print("#############################")
print("Best K =", best_K)
print("Applying K-means")
best_cluster = KMeans(n_clusters=best_K, n_jobs=-1)
best_cluster_result = best_cluster.fit(X)
print("Finished")
print("#############################")    

#############################
Best K = 61
Applying K-means
Finished
#############################


In [9]:
y_pred = best_cluster_result.labels_
arrays=[]
for i in range(best_K):
    dist = np.linalg.norm(best_cluster_result.cluster_centers_[i]-npX[y_pred==i],axis=1)
    index = np.argmin(dist)
    arrays.append(npX[y_pred==i][index])   

medoids = np.vstack(arrays)
print(medoids.shape)
print(medoids)

(61, 733)
[[-0.02084702  0.00864816 -0.02207969 ... -0.00428908 -0.00508571
   0.01500977]
 [ 0.13959183 -0.21770751 -0.11784142 ...  0.02807976  0.01087402
   0.01806026]
 [ 0.33406802  0.02380238  0.12272257 ...  0.01926641  0.00619181
   0.05321628]
 ...
 [-0.02140241 -0.01677267 -0.02584795 ...  0.04212087 -0.03687889
  -0.03582874]
 [-0.01382887 -0.01317931 -0.02836163 ...  0.01689735  0.01340904
  -0.00068102]
 [ 0.00650213  0.37024835 -0.12173441 ...  0.00950344  0.01638051
  -0.0043304 ]]


In [10]:
l = []
for i in range(best_K):
    l.append(np.where(np.all(npX==medoids[i],axis=1))[0][0]) # these last [0][0] return the index, since the output of this command is (array([id]),)
    
medoids_index = np.array(l)
print(medoids_index)
print(len(np.unique(medoids_index)))

[ 7728 10512  9602  6330  6494   867  4205  9215  7082 10559  4317 12132
  2785  9845   753  7202  2258  7884  6849  9915  3820 11004  9470  2212
  6063 12418  4026   197  7125 12440  9558 11975 12493  5875 10380  2632
  5053  1535  6089  4691  9963  5710  3289 12650 10151  6074 12642  2885
 12601 12560 10440  4536  3189  6221  9302  1055 12778  1123  2806 12891
 10182]
61


## Selecting medoids neighbors
Here the medoids neighbors are selected in order to plot them and see if the clusters make sense.

In [11]:
nm = 5 #number of medoids to be analyzed
nn = 5 #number of neighbor of each medoids
np.random.seed(42)
selected_medoids_index = np.random.randint(0,best_K,nm)
selected_medoids = medoids[selected_medoids_index]
print("Random indexes:",selected_medoids_index)
indexes_per_medoid = []

for i in range(nm):
    p = selected_medoids[i]
    aux = npX[y_pred==selected_medoids_index[i]]
    d = np.linalg.norm(aux-p,axis=1)
    points = []
    points.append(medoids_index[selected_medoids_index[i]])
    
    for j in range(nn):
        min_d = d.argmin()
        while (d[min_d] == 0.):
            d[min_d] = np.inf
            min_d = d.argmin()
        points = points + (np.where(np.all(npX==aux[min_d],axis=1))[0]).tolist()
        d[min_d] = np.inf
        
    indexes_per_medoid.append(np.unique(points).tolist())

    
print("Points:",indexes_per_medoid)
print(len(indexes_per_medoid))

Random indexes: [38 51 28 14 42]
Points: [[6089, 6233, 6295, 6345, 6395, 6444, 6500, 6665, 6731, 6798, 6946, 6998, 7049, 7106, 7247, 7309, 7437, 7546, 7596, 7647, 7702, 7753, 7808, 7949, 8014, 8064, 8115, 8179, 8312, 8372, 8436, 8485, 8534, 8583, 8639, 8874, 8937, 9000, 9071, 9158, 9208, 9270], [4536, 9997, 10415, 10419, 11328, 11331], [3060, 7125, 7138, 7638, 8515, 12108], [588, 753, 1397, 4131, 8450, 8698], [2894, 3289, 10197, 12185, 12477, 13214]]
5


## Analyzing the Medoids
Here we start the analysis of the medoids and their neighbors.

### Exporting the news of the medoids and clusters

In [12]:
import os 

os.makedirs('../output/clusters_PCA/', exist_ok=True)
os.makedirs('../output/medoids_PCA/', exist_ok=True)


print("Random indexes:",selected_medoids_index)
for i in range(best_K):
    np.savetxt("../output/clusters_PCA/k_"+str(i)+".txt", news[y_pred==i].values, fmt='%s', encoding='utf8')
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    np.savetxt("../output/medoids_PCA/k_"+str(selected_medoids_index[i])+".txt", np.take(news,p,0).values, fmt='%s')

Random indexes: [38 51 28 14 42]


### News of the medoids and their neighbors

In [13]:
for p in indexes_per_medoid:
    info = np.take(news,p,0).loc[:,"headline_test"].values
    print("----------------------------")
    for i in range(info.size):
        print(info[i],"\n")
print("----------------------------")

----------------------------
Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out! 

Everyday Health Daily Digest is out

### Generating cloud of words

In [14]:
%matplotlib inline
from wordcloud import WordCloud
import matplotlib
import matplotlib.pyplot as plt

os.makedirs('../output/wordcloud_clusters_PCA/', exist_ok=True)
os.makedirs('../output/wordcloud_medoids_PCA/', exist_ok=True)

# clusters
for i in range(best_K):
    info = ' '.join(news[y_pred==i].loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/wordcloud_clusters_PCA/k_"+str(i))
    plt.close(fig)

#medoids
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    info = ' '.join(np.take(news,p,0).loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/wordcloud_medoids_PCA/k_"+str(selected_medoids_index[i]))
    plt.close(fig)
